### Step-1: Preliminary setup

In [163]:
#Import gensim library
import gensim
from gensim.models import Word2Vec
from nltk.corpus import PlaintextCorpusReader
import nltk

### Step-2: Load Dataset and Train Word2Vec model

In [164]:
#Load reviews files from the following cities : Chicago, San Francisco, New York City
tokensents_lst=[]
cities = ["chicago", "san-francisco","new-york-city"]
for city in cities:
    print(city)
    corpus_root = 'OpinRankDataset/hotels/'+city
    filelists = PlaintextCorpusReader(corpus_root, '.*', encoding='latin-1')
    tokensents = filelists.sents() # sentence-splitted and word-tokenized list of sentences
    tokensents_lst += list(tokensents) #Concatente all the sentence tokens into one list
#Train the Word2Vec Model
model = Word2Vec(tokensents_lst)    

chicago
san-francisco
new-york-city


### Step-3: Obtaining Feature Words

In [ ]:
#Embeding for "service" word
#print(model.wv['Rooms'])

In [165]:
#Similar word vector as the word "service".
#HW5: 'room' (top 50), 'rooms' (top 50), 'Room' (top 20) and 'Rooms' (top 20)

#Initialize the list of feature
feature_lst= []
for word in ["room","rooms", "Room", "Rooms"]:
    if word[0]=="r": #Find top 50 
        word_lst = list(model.most_similar(word, topn=50))
        for i in range(len(word_lst)):
            #print(word_lst[i][0])
            feature_lst.append(word_lst[i][0])
    else:
        word_lst = list(model.most_similar(word, topn=20))
        for i in range(len(word_lst)):
            #print(word_lst[i][0])
            feature_lst.append(word_lst[i][0])
print(set(feature_lst))            
feature_lst = list(set(feature_lst))            

{'Meals', 'mirrors', 'Accomodations', 'bed', 'TVs', 'television', 'accomodations', 'Bedroom', 'balcony', 'sinks', 'guestroom', 'suitcase', 'coffeemaker', 'bedrooms', 'fixtures', 'Toiletries', 'crib', 'Breakfasts', 'windows', 'disappointingly', 'cockroach', 'suites', 'towers', 'settings', 'buildings', 'alcove', 'lifts', 'Bathroom', 'wardrobe', 'incidents', 'baths', 'armoire', 'Bathrooms', 'washrooms', 'Bedding', 'surprisingly', 'sofabed', 'balconies', 'ones', 'Pillows', 'dresser', 'cot', 'Bedrooms', 'elevators', 'unit', 'fireplaces', 'Furnishings', 'bathroom', 'Lobby', 'bathtubs', 'rooms', 'Beds', 'Bed', 'guestrooms', 'accommodations', 'bedroom', 'frig', 'cupboard', 'sheets', 'apartments', 'furnishings', 'showerheads', 'Suite', 'headboard', 'units', 'Pool', 'oom', 'mirror', 'tub', 'king', 'room', 'refrigerator', 'Gym', 'bathtub', 'table', 'Showers', 'washroom', 'vase', 'rm', 'cabinets', 'Rates', 'closets', 'showerhead', 'Hallways', 'terraces', 'beds', 'closet', 'window', 'refridgerator'

/Users/KevQuant/anaconda/lib/python3.6/site-packages/ipykernel_launcher.py:8: DeprecationWarning: Call to deprecated `most_similar` (Method will be removed in 4.0.0, use self.wv.most_similar() instead).
  
/Users/KevQuant/anaconda/lib/python3.6/site-packages/ipykernel_launcher.py:13: DeprecationWarning: Call to deprecated `most_similar` (Method will be removed in 4.0.0, use self.wv.most_similar() instead).
  del sys.path[0]


### Step-4: Extract opinion segments from reviews

In [78]:
#sent = nltk.pos_tag(nltk.word_tokenize("She girl let down her long hair"))
#sent = nltk.pos_tag(nltk.word_tokenize("The large rooms seemed pretty clean"))
#sent=nltk.pos_tag(nltk.word_tokenize("the room is clean"))
#sent=nltk.pos_tag(nltk.word_tokenize("the bathrooms are spacious"))
#sent=nltk.pos_tag(nltk.word_tokenize("her pretty large room"))
#sent=nltk.pos_tag(nltk.word_tokenize("very clean room"))
#sent=nltk.pos_tag(nltk.word_tokenize("modern shower head"))
#sent=nltk.pos_tag(nltk.word_tokenize("room was spotless"))
sent=nltk.pos_tag(nltk.word_tokenize("modern shower head"))
print(sent)

[('modern', 'JJ'), ('shower', 'JJR'), ('head', 'NN')]


In [106]:
nltk.word_tokenize("modern shower head")

['modern', 'shower', 'head']

In [158]:
sent=nltk.pos_tag(nltk.word_tokenize("The bathroom was spacious, well equiped and clean"))

#regex grammar
grammar = r"""
NP:{<DT|PRP\$>?<RB>*<JJ.*>+<NN.*>}
p1:{<RB.*>*<VB.*>*<RB.*>*<JJ.*>*}
clause1:{<DT>*<NN.*><,|CC>*<NN.*>*<p1>*<,|CC>*<p1>*<,|CC>*<p1>*}
"""
#NLTK Regex parser
cp = nltk.RegexpParser(grammar)

test_root = 'OpinRankDataset/hotels/chicago/'
testfile= test_root  + "usa_illinois_chicago_hard_rock_hotel_chicago"
f= open(testfile, "r",encoding='latin-1')
text = f.read()
sentences = nltk.sent_tokenize(text)
sentlist=[nltk.word_tokenize(sent) for sent in sentences]


review_lst=[]
for sent in sentlist:
    tree = cp.parse(nltk.pos_tag(sent))
    #print(tree)
    for subtree in tree.subtrees():
        #print(subtree.leaves())
        #print(subtree)
        if subtree.label() =="NP":
            print("NP")
            print(subtree.leaves())
        if subtree.label() =="clause1":
            print("clause1")
            print(subtree.leaves())
            review_lst.append(subtree.leaves())


NP
[('Fantastic', 'JJ'), ('Experience', 'NNP')]
clause1
[('Great', 'NNP'), ('Location', 'NN')]
NP
[('a', 'DT'), ('sleek', 'JJ'), ('exterior', 'NN')]
NP
[('cool', 'JJ'), ('vibe', 'NN')]
clause1
[('room', 'NN'), ('was', 'VBD')]
NP
[('sooo', 'JJ'), ('cool', 'NN')]
clause1
[('chrome', 'NN'), (',', ','), ('lots', 'NNS')]
clause1
[('mirrors', 'NNS'), (',', ',')]
NP
[('a', 'DT'), ('wonderfully', 'RB'), ('comfortable', 'JJ'), ('bed', 'NN')]
clause1
[('pillows', 'NNS'), (',', ','), ('and', 'CC')]
NP
[('great', 'JJ'), ('bathroom', 'NN')]
clause1
[('facilities', 'NNS'), (',', ',')]
clause1
[('built', 'NN')]
clause1
[('a', 'DT'), ('staff', 'NN')]
clause1
[('a', 'DT'), ('room', 'NN')]
clause1
[('quot', 'NN')]
clause1
[('a', 'DT'), ('bit', 'NN')]
clause1
[('a', 'DT'), ('view', 'NN'), ('&', 'CC'), ('quot', 'NN')]
clause1
[('a', 'DT'), ('view', 'NN')]
clause1
[('Michigan', 'NNP'), ('Avenue', 'NNP'), (',', ',')]
clause1
[('the', 'DT'), ('Wrigley', 'NNP'), ('Plaza', 'NNP'), (',', ','), ('and', 'CC')]
cl

NP
[('a', 'DT'), ('little', 'JJ'), ('dated', 'JJ'), ('\x96', 'NN')]
clause1
[('nothing', 'NN')]
clause1
[('the', 'DT'), ('television', 'NN')]
clause1
[('an', 'DT'), ('LCD', 'NNP'), ('and', 'CC')]
clause1
[('the', 'DT'), ('walls', 'NNS')]
clause1
[('others', 'NNS'), ('have', 'VBP'), ('pointed', 'VBN')]
clause1
[('the', 'DT'), ('rooms', 'NNS'), ('are', 'VBP')]
NP
[('a', 'DT'), ('little', 'JJ'), ('dark', 'NN')]
clause1
[('Didn\x92t', 'NNP'), ('try', 'VBD')]
clause1
[('the', 'DT'), ('food', 'NN')]
clause1
[('the', 'DT'), ('hotel', 'NN')]
clause1
[('plenty', 'NN')]
NP
[('nice', 'JJ'), ('restaurants', 'NNS')]
NP
[('nearby.The', 'JJ'), ('location', 'NN')]
clause1
[('the', 'DT'), ('hotel', 'NN'), ('was', 'VBD'), ('perfect', 'JJ')]
clause1
[('distance', 'NN')]
clause1
[('a', 'DT'), ('lot', 'NN')]
NP
[('the', 'DT'), ('touristy', 'JJ'), ('things', 'NNS')]
clause1
[('Chicago', 'NNP')]
clause1
[('The', 'DT'), ('view', 'NN')]
clause1
[('room', 'NN'), ('wasn\x92t', 'NN'), ('great', 'JJ'), ('but', 'CC

[('Rock', 'NNP'), ('Hotels', 'NNP')]
clause1
[('the', 'DT'), ('Hard', 'NNP'), ('Rock', 'NNP')]
clause1
[('Orlando', 'NNP')]
clause1
[('February', 'NNP'), ('and', 'CC')]
clause1
[('son', 'NN'), ('and', 'CC'), ('husband', 'NN')]
clause1
[('this', 'DT'), ('program', 'NN'), ('there', 'RB')]
clause1
[('well.Thanks', 'NNS')]
NP
[('a', 'DT'), ('great', 'JJ'), ('time', 'NN')]
clause1
[('Poor', 'NNP'), ('room', 'NN'), ('dirty', 'NN'), ('bathroom', 'NN')]
NP
[('worn', 'JJ'), ('washcloths', 'NNS')]
NP
[('inoperative', 'JJ'), ('elevators', 'NNS')]
NP
[('The', 'DT'), ('check-in', 'JJ'), ('process', 'NN')]
NP
[('the', 'DT'), ('only', 'JJ'), ('part', 'NN')]
clause1
[('the', 'DT'), ('stay', 'NN')]
clause1
[('room', 'NN'), ('was', 'VBD'), ('not', 'RB'), ('clean', 'JJ')]
clause1
[('trash', 'NN')]
NP
[('a', 'DT'), ('previous', 'JJ'), ('guest', 'NN')]
clause1
[('the', 'DT'), ('chair', 'NN')]
clause1
[('the', 'DT'), ('corner', 'NN')]
clause1
[('the', 'DT'), ('room', 'NN')]
clause1
[('\x93rings\x94', 'NNS')

clause1
[('this', 'DT'), ('darkness', 'NN')]
clause1
[('the', 'DT'), ('hotel', 'NN')]
clause1
[('room', 'NN'), ('or', 'CC')]
clause1
[('the', 'DT'), ('fitness', 'NN'), ('center', 'NN')]
NP
[('romantic.The', 'JJ'), ('China', 'NNP')]
clause1
[('Grill', 'NNP'), ('is', 'VBZ')]
NP
[('a', 'DT'), ('top', 'JJ'), ('notch', 'NN')]
clause1
[('restaurant', 'NN')]
clause1
[('the', 'DT'), ('hotel', 'NN')]
clause1
[('the', 'DT'), ('lobby', 'NN')]
NP
[('a', 'DT'), ('fine', 'JJ'), ('meal', 'NN')]
clause1
[('a', 'DT'), ('coat', 'NN'), ('or', 'CC'), ('find', 'VB')]
clause1
[('thing', 'NN')]
clause1
[('no', 'DT'), ('notification', 'NN')]
clause1
[('room', 'NN')]
clause1
[('packages', 'NNS'), ('had', 'VBD'), ('arrived', 'VBN')]
clause1
[('a', 'DT'), ('paper', 'NN'), ('notice', 'NN'), ('or', 'CC'), ('had', 'VBD')]
NP
[('the', 'DT'), ('light', 'JJ'), ('blink', 'NN')]
clause1
[('phone', 'NN')]
clause1
[('any', 'DT'), ('packages', 'NNS'), ('had', 'VBD'), ('arrived', 'VBN')]
NP
[('an', 'DT'), ('annoying', 'JJ')

clause1
[('The', 'DT'), ('John', 'NNP'), ('Entwistle', 'NNP'), ('theme', 'NN')]
clause1
[('floor', 'NN'), ('was', 'VBD')]
NP
[('a', 'DT'), ('nice', 'JJ'), ('touch', 'NN')]
NP
[('single', 'JJ'), ('fathers', 'NNS')]
clause1
[('a', 'DT'), ('fun', 'NN')]
NP
[('affordable', 'JJ'), ('vacation', 'NN')]
clause1
[('children', 'NNS'), ('and', 'CC')]
clause1
[('The', 'DT'), ('Hard', 'NNP'), ('Rock', 'NNP'), ('Hotel', 'NNP')]
clause1
[('Chicago', 'NNP'), ('really', 'RB'), ('helped', 'VBD')]
clause1
[('Was', 'NNP'), ('Nice', 'NNP'), ('Pretty', 'NNP'), ('hotel', 'NN')]
NP
[('good', 'JJ'), ('service', 'NN')]
NP
[('front', 'JJ'), ('desk', 'NN')]
clause1
[('fair', 'NN')]
clause1
[('the', 'DT'), ('restaurant', 'NN')]
clause1
[('The', 'DT'), ('people', 'NNS')]
clause1
[('the', 'DT'), ('meetings', 'NNS'), ('department', 'NN'), ('were', 'VBD'), ('very', 'RB'), ('good', 'JJ')]
clause1
[('The', 'DT'), ('banquet', 'NN')]
NP
[('a', 'DT'), ('good', 'JJ'), ('hotel', 'NN')]
clause1
[('the', 'DT'), ('money', 'NN')

...

clause1
[('THANK', 'NNP'), ('YOU', 'NNP'), ('HRH', 'NNP')]
NP
[('a', 'DT'), ('nice', 'JJ'), ('end', 'NN')]
clause1
[('vacation', 'NN')]
NP
[('The', 'DT'), ('only', 'JJ'), ('odd', 'JJ'), ('thing', 'NN')]
clause1
[('the', 'DT'), ('room', 'NN'), ('was', 'VBD')]
clause1
[('the', 'DT'), ('bathroom', 'NN')]
clause1
[('the', 'DT'), ('configuration', 'NN')]
clause1
[('the', 'DT'), ('door', 'NN')]
clause1
[('the', 'DT'), ('door', 'NN'), ('was', 'VBD'), ('opened', 'VBN')]
clause1
[('the', 'DT'), ('vanity', 'NN'), ('counter', 'NN'), ('and', 'CC')]
clause1
[('the', 'DT'), ('commode', 'NN')]
NP
[('a', 'DT'), ('big', 'JJ'), ('problem', 'NN')]
NP
[('a', 'DT'), ('fluffy', 'JJ'), ('cotton', 'NN')]
clause1
[('rug', 'NN')]
clause1
[('the', 'DT'), ('floor', 'NN'), ('and', 'CC')]
clause1
[('the', 'DT'), ('way', 'NN')]
clause1
[('the', 'DT'), ('door', 'NN'), ('and', 'CC')]
clause1
[('an', 'DT'), ('inconvenience', 'NN')]
NP
[('The', 'DT'), ('shower', 'JJR'), ('stall', 'NN')]
clause1
[('HUGE', 'NNP')]
clause1

clause1
[('The', 'DT'), ('decor', 'NN'), (',', ',')]
NP
[('a', 'DT'), ("rock'n", 'JJ'), ('roll/hip', 'NN')]
clause1
[('element', 'NN'), (',', ','), ('are', 'VBP'), ('contempory', 'JJ'), ('and', 'CC'), ('very', 'RB'), ('comfortable', 'JJ')]
clause1
[('The', 'DT'), ('service', 'NN'), ('is', 'VBZ'), ('excellent', 'JJ')]
clause1
[('chicago', 'NN')]
NP
[('desperado', 'JJ'), ('destination', 'NN')]
clause1
[('the', 'DT'), ('BOLD', 'NNP')]
NP
[('new', 'JJ'), ('michigan', 'NNP')]
clause1
[('hotel', 'NN'), ('destination', 'NN')]
NP
[('namely', 'RB'), ('hard', 'JJ'), ('rock', 'NN')]
clause1
[('hotel', 'NN')]
clause1
[('the', 'DT'), ('experience', 'NN'), ('immediately', 'RB')]
clause1
[('hip', 'NN'), (',', ','), ('atmospherically', 'RB'), ('relaxing', 'VBG')]
NP
[('simple', 'JJ'), ('undertones', 'NNS')]
clause1
[('elegance', 'NN'), (',', ',')]
NP
[('a', 'DT'), ('mindful', 'JJ'), ('valet', 'NN')]
clause1
[('amp', 'NN')]
clause1
[('baggage', 'NN'), ('handler', 'NN'), (',', ',')]
NP
[('desk', 'JJ'), 

clause1
[('the', 'DT'), ('rest', 'NN')]
clause1
[('a', 'DT'), ('picture', 'NN')]
clause1
[('a', 'DT'), ('guy', 'NN')]
NP
[('the', 'DT'), ('same', 'JJ'), ('room', 'NN')]
clause1
[('the', 'DT'), ('bathroom', 'NN')]
clause1
[('The', 'DT'), ('air', 'NN'), ('vent', 'NN'), ('made', 'VBD'), ('noise', 'RB')]
clause1
[('all', 'DT'), ('night', 'NN'), ('long', 'RB')]
clause1
[('Room', 'NNP'), ('service', 'NN'), ('took', 'VBD'), ('forever', 'RB')]
clause1
[('bucks', 'NNS')]
clause1
[('a', 'DT'), ('day', 'NN'), ('just', 'RB')]
clause1
[('the', 'DT'), ('car', 'NN')]
clause1
[('a', 'DT'), ('parking', 'NN'), ('garage', 'NN')]
clause1
[('something', 'NN')]
clause1
[('the', 'DT'), ('bed', 'NN'), ('and', 'CC')]
clause1
[('the', 'DT'), ('bathroom', 'NN')]
clause1
[('girlfriend', 'NN'), ('thought', 'NN')]
clause1
[('sprinklers', 'NNS')]
clause1
[('sprinklers', 'NNS')]
NP
[('the', 'DT'), ('main', 'JJ'), ('room', 'NN')]
clause1
[('the', 'DT'), ('Bathroom', 'NN')]
clause1
[('Cold', 'NN')]
clause1
[('this', 'D

NP
[('the', 'DT'), ('only', 'JJ'), ('ones', 'NNS')]
clause1
[('the', 'DT'), ('drinks', 'NNS')]
clause1
[('The', 'DT'), ('room', 'NN'), ('was', 'VBD'), ('entirely', 'RB'), ('gray', 'JJ')]
NP
[('gray', 'JJ'), ('walls', 'NNS')]
NP
[('gray', 'JJ'), ('bedspread', 'NN')]
NP
[('gray', 'JJ'), ('tv', 'NN')]
clause1
[('everything', 'NN'), ('was', 'VBD'), ('clean', 'JJ'), ('and', 'CC')]
clause1
[('the', 'DT'), ('employees', 'NNS'), ('were', 'VBD'), ('nice', 'JJ')]
clause1
[('the', 'DT'), ('location', 'NN'), ('was', 'VBD'), ('great', 'JJ')]
clause1
[('Wow', 'NN')]
NP
[('a', 'DT'), ('great', 'JJ'), ('hotel', 'NN')]
clause1
[('Everything', 'NN'), ('worked', 'VBD')]
NP
[('A', 'DT'), ('constant', 'JJ'), ('party', 'NN')]
clause1
[('atmosphere', 'NN')]
clause1
[('Sheer', 'NNP'), ('pleasure', 'NN')]
clause1
[('Bravo', 'NN')]
clause1
[('Hard', 'NNP'), ('Rock', 'NNP'), ('Made', 'NNP'), ('grat', 'NN'), ('impression', 'NN')]
clause1
[('Chicago', 'NNP')]
clause1
[('the', 'DT'), ('weekend', 'NN')]
clause1
[('n

clause1
[('Will', 'NNP'), ('stay', 'VB'), ('here', 'RB'), ('again', 'RB')]
clause1
[('Stayed', 'NNP'), ('here', 'RB')]
clause1
[('night', 'NN')]
clause1
[('a', 'DT'), ('group', 'NN')]
clause1
[('bachelor', 'NN'), ('party', 'NN'), ('night', 'NN')]
clause1
[('the', 'DT'), ('town', 'NN')]
clause1
[('choice', 'NN')]
clause1
[('hotel', 'NN')]
clause1
[('The', 'DT'), ('location', 'NN'), ('was', 'VBD'), ('excellent', 'JJ'), (',', ',')]
clause1
[('the', 'DT'), ('place', 'NN'), ('was', 'VBD'), ('very', 'RB'), ('clean', 'JJ'), ('and', 'CC')]
clause1
[('the', 'DT'), ('rooms', 'NNS'), ('were', 'VBD'), ('suprisingly', 'RB'), ('bigger', 'JJR')]
clause1
[('the', 'DT'), ('staff', 'NN'), ('was', 'VBD'), ('friendly', 'JJ'), ('and', 'CC'), ('professional', 'JJ'), (',', ',')]
clause1
[('the', 'DT'), ('wait', 'NN')]
clause1
[('car', 'NN')]
clause1
[('minutes', 'NNS')]
clause1
[('in/out', 'NN'), ('privileges', 'NNS'), ('is', 'VBZ')]
NP
[('a', 'DT'), ('pretty', 'RB'), ('good', 'JJ'), ('deal', 'NN')]
clause1


In [167]:
review_lst2=[]
for line in review_lst[0:10]:
    for item in line:
        if item[0] in feature_lst:
            review_lst2.append(line)
            continue
review_lst2            

[[('room', 'NN'), ('was', 'VBD')],
 [('mirrors', 'NNS'), (',', ',')],
 [('a', 'DT'), ('room', 'NN')]]

In [154]:
import numpy as np
import pandas as pd

review_lst[0][1][0]

IndexError: list index out of range

### Step-5: Parse all reviews in the city "Chicago"

In [105]:
import nltk

test_root = 'OpinRankDataset/hotels/chicago/'
testfile= test_root  + "usa_illinois_chicago_hard_rock_hotel_chicago"
f= open(testfile, "r",encoding='latin-1')
text = f.read()
sentences = nltk.sent_tokenize(text)
sentlist=[nltk.word_tokenize(sent) for sent in sentences]
print(sentlist[50])


['or', 'close', 'the', 'waterproof', 'drapes', '.']


[['Nov', '25', '2009', 'Fantastic', 'Experience', 'Great', 'Location', '!'], ['What', 'a', 'sleek', 'exterior', ',', 'cool', 'vibe', ',', 'and', 'yet', 'so', 'laid', 'back', 'and', 'friendly', '!'], ['Our', 'room', 'was', 'sooo', 'cool', ',', 'with', 'chrome', ',', 'lots', 'of', 'mirrors', ',', 'a', 'wonderfully', 'comfortable', 'bed', 'and', 'pillows', ',', 'and', 'great', 'bathroom', 'facilities', ',', 'with', 'a', 'really', 'roomy', 'shower', '-', 'built', 'for', '2', '?', '.'], ['And', 'what', 'a', 'staff', '!'], ['We', 'asked', 'for', 'a', 'room', 'with', '&', 'quot', ';', 'a', 'bit', 'of', 'a', 'view', '&', 'quot', ';', 'and', 'wound', 'up', 'with', 'a', 'view', 'of', 'Michigan', 'Avenue', ',', 'the', 'Wrigley', 'Plaza', ',', 'and', 'the', 'river', '.'], ['And', 'the', 'location', '!'], ['If', 'you', 'were', 'to', 'make', 'downtown', 'Chicago', 'into', 'a', 'dart', 'board', ',', 'this', 'would', 'be', 'the', 'bullseye', '.'], ['It', 'is', 'absolutely', 'dead-center', ',', 'right'

In [130]:
#sent = nltk.pos_tag(nltk.word_tokenize("She girl let down her long hair"))
sent = nltk.pos_tag(nltk.word_tokenize("quick"))
print(sent)

#regex grammar
grammar = r"""
NP:{<DT|PRP\$>?<JJ>*<NN>} 
FOO:{<NP><VB.><JJ>}"""
#"NP:{DT|PRP\$>?<JJ>*<NN>}" #, FOO: {<NP><VB.><JJ>}"

#NLTK Regex parser
cp = nltk.RegexpParser(grammar)
tree = cp.parse(sent)
print(tree)
for subtree in tree.subtrees():
    print(subtree)



[('quick', 'NN')]


In [56]:
#NLTK Regex parser
cp = nltk.RegexpParser(grammar)
tree = cp.parse(sent)
print(tree)
for subtree in tree.subtrees():
    print(subtree)


(S She/PRP girl/VBD let/VB down/RP her/PRP$ (NP long/JJ hair/NN))
(S She/PRP girl/VBD let/VB down/RP her/PRP$ (NP long/JJ hair/NN))
(NP long/JJ hair/NN)


In [132]:
for subtree in tree.subtrees():
    if subtree.label() =="NP":
        print(subtree.leaves())

[('quick', 'NN')]


In [ ]:
for sent in sentlist:
    cp.parse(nltk.pos_tag(sent))
    for subtree in tree.subtrees():
        if subtree.label() =="NP":
            print(subtree.leaves())